In [1]:
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
import pickle
import re

In [2]:
data = open("Metamorphosis.txt").read()
tokenizer = Tokenizer()
corpus = data.lower().split("\n")
print(len(corpus))
tokenizer.fit_on_texts(corpus)
# print(tokenizer.word_index)
tot_words = len(tokenizer.word_index)+1
# create input sequences
input_sequences =[]
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# print(input_sequences)



# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])

input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences.shape

2221


(23144, 18)

In [3]:
print(tokenizer.word_index)

{'the': 1, 'to': 2, 'and': 3, 'he': 4, 'of': 5, 'his': 6, 'was': 7, 'in': 8, 'it': 9, 'had': 10, 'a': 11, 'that': 12, 'as': 13, 'with': 14, 'not': 15, 'she': 16, 'for': 17, 'gregor': 18, 'him': 19, 'her': 20, 'would': 21, 'at': 22, 'but': 23, 'on': 24, 'they': 25, 'all': 26, 'this': 27, 'be': 28, 'from': 29, 'room': 30, 'if': 31, 'could': 32, 'you': 33, 'or': 34, 'out': 35, 'have': 36, 'there': 37, 'been': 38, 'gregor’s': 39, 'so': 40, 'sister': 41, 'father': 42, 'gutenberg': 43, 'project': 44, 'himself': 45, 'door': 46, 'any': 47, 'up': 48, 'back': 49, 'by': 50, 'even': 51, 'now': 52, 'then': 53, 'no': 54, 'into': 55, 'mother': 56, 'one': 57, 'work': 58, 'about': 59, 'more': 60, 'did': 61, 'their': 62, 'when': 63, '”': 64, 'were': 65, 'them': 66, 'only': 67, 'way': 68, 'do': 69, 'other': 70, 'than': 71, 'time': 72, 'what': 73, 'tm': 74, 'i': 75, 'little': 76, 'said': 77, 'just': 78, 'is': 79, 'without': 80, '1': 81, 'still': 82, 'first': 83, 'get': 84, 'see': 85, 'some': 86, 'made': 8

In [4]:
print(input_sequences[0])
# print(input_sequences.shape)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  57 167]


In [5]:
# on labels one-hot-encoding is done 
X,y = input_sequences[:,:-1], input_sequences[:,-1]
print(y.shape)
y= to_categorical(y, num_classes = tot_words)

(23144,)


In [6]:
print(X.shape)
y.shape

(23144, 17)


(23144, 3222)

In [7]:
model = Sequential()
model.add(Embedding(tot_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(tot_words/2, activation='relu')) #dense layer including regularization
model.add(Dense(tot_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics = ['accuracy']) 
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 17, 100)           322200    
_________________________________________________________________
bidirectional (Bidirectional (None, 17, 300)           301200    
_________________________________________________________________
dropout (Dropout)            (None, 17, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 1611)              162711    
_________________________________________________________________
dense_1 (Dense)              (None, 3222)              5193864   
Total params: 6,140,375
Trainable params: 6,140,375
Non-trainable params: 0
______________________________________________

In [8]:
# prev 30
history = model.fit(X,y,epochs=100, verbose=1)

Train on 23144 samples
Epoch 1/100
23144/23144 [==============================] - 32s 1ms/sample - loss: 6.2669 - accuracy: 0.0556
Epoch 2/100
23144/23144 [==============================] - 28s 1ms/sample - loss: 5.8383 - accuracy: 0.0617
Epoch 3/100
23144/23144 [==============================] - 27s 1ms/sample - loss: 5.5468 - accuracy: 0.0868
Epoch 4/100
23144/23144 [==============================] - 27s 1ms/sample - loss: 5.2676 - accuracy: 0.1123
Epoch 5/100
23144/23144 [==============================] - 27s 1ms/sample - loss: 5.0581 - accuracy: 0.1260
Epoch 6/100
23144/23144 [==============================] - 27s 1ms/sample - loss: 4.9060 - accuracy: 0.1369
Epoch 7/100
23144/23144 [==============================] - 27s 1ms/sample - loss: 4.7747 - accuracy: 0.1456
Epoch 8/100
23144/23144 [==============================] - 27s 1ms/sample - loss: 4.6500 - accuracy: 0.1544
Epoch 9/100
23144/23144 [==============================] - 27s 1ms/sample - loss: 4.5294 - accuracy: 0.1574
Epoch

23144/23144 [==============================] - 27s 1ms/sample - loss: 0.5062 - accuracy: 0.8468
Epoch 77/100
23144/23144 [==============================] - 27s 1ms/sample - loss: 0.4968 - accuracy: 0.8496
Epoch 78/100
23144/23144 [==============================] - 27s 1ms/sample - loss: 0.4906 - accuracy: 0.8505
Epoch 79/100
23144/23144 [==============================] - 27s 1ms/sample - loss: 0.4756 - accuracy: 0.8553
Epoch 80/100
23144/23144 [==============================] - 27s 1ms/sample - loss: 0.4656 - accuracy: 0.8577
Epoch 81/100
23144/23144 [==============================] - 27s 1ms/sample - loss: 0.4666 - accuracy: 0.8583
Epoch 82/100
23144/23144 [==============================] - 27s 1ms/sample - loss: 0.4637 - accuracy: 0.8565
Epoch 83/100
23144/23144 [==============================] - 27s 1ms/sample - loss: 0.4683 - accuracy: 0.8554
Epoch 84/100
23144/23144 [==============================] - 27s 1ms/sample - loss: 0.4424 - accuracy: 0.8665
Epoch 85/100
23144/23144 [======

In [9]:
# history = model.fit(X,y,epochs=100, verbose=1)

In [10]:
model_json = model.to_json()
with open("model_bilstm_19_04.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model_bilstm_19_04.h5")
print("Saved model to disk")

Saved model to disk


In [11]:
def plot_graph(history, string):
    plt.plot(history.history(string))
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.show()
  

In [12]:
# load json and create model

import json
from tensorflow.keras.models import model_from_json
json_file = open('model_bilstm_19_04.json','r')
loaded_model_json= json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model_bilstm_19_04.h5")
print("Loaded model from disk")

Loaded model from disk


In [13]:
model = loaded_model

In [14]:
seed_text = "He began running"
next_word= 6
for _ in range(next_word):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list= (pad_sequences([token_list],maxlen=max_sequence_len-1, padding="pre"))
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if(index==predicted):
            output_word=word
            print(output_word)
            break
    seed_text+=(" " + output_word)
print(seed_text)

a
broom
but
then
he
stood
He began running a broom but then he stood


In [ ]:
def prepare_input(text):
    x = np.zeros((1, length_word, len(unique_words)))
    for ind,word in enumerate(text.split()):
    x[0, ind, words_index[word]] = 1
    return x
prepare_input("It is not a lack".lower())


def sample(predicts, n_top_words=5):
    predict = np.asarray(predicts).astype('float64')
    predicts = np.log(predicts)
    exp_preds = np.exp(predicts)
    predicts = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(n_top_words, range(len(predicts)), predicts.take)



def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    print(x)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)

    return [unique_words[idx] for idx in next_indices]


# q =  "One night—it was on the twentieth of March"
q =  "I want to be in past days"
q =  "It was a quarter past six when we left Baker Street"
q =  "He catch the train every"
q =  "But I understand that all the other clothes were found in the room"

print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:4])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))